In [8]:
#!pip install category_encoders

     |████████████████████████████████| 81 kB 755 kB/s eta 0:00:01


In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

import category_encoders as ce
from feature_engineering import encoding


### Кодирование признаков


#### Зачем важно кодирование признаков?

Мы должны преобразовать строки категориальных переменных в числа, чтобы алгоритмы машинного обучения могли обрабатывать эти значения. Даже если вы видите, что алгоритм может принимать категориальные входные данные, скорее всего, алгоритм включает в себя процесс кодирования.

## Загрузка данных

In [2]:
use_cols = [
    'Pclass', 'Sex', 'Age', 'Fare', 'SibSp',
    'Survived'
]

data = pd.read_csv('./data/titanic.csv', usecols=use_cols)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,male,22.0,1,7.2500
1,1,1,female,38.0,1,71.2833
2,1,3,female,26.0,0,7.9250
3,1,1,female,35.0,1,53.1000
4,0,3,male,35.0,0,8.0500


In [3]:
X_train, X_test, y_train, y_test = train_test_split(data, data.Survived, test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((623, 6), (268, 6))

## One-hot encoding

Замена категориального признака 1 или 0, если данная категория верна для объекта. Чем больше категорий, тем больше новых признаков.

In [4]:
pd.get_dummies(X_train, drop_first=True)

,Survived,Pclass,Age,SibSp,Fare,Sex_male
857,1,1,51.0,0,26.5500,1
52,1,1,49.0,1,76.7292,0
386,0,3,1.0,5,46.9000,1
124,0,1,54.0,0,77.2875,1
578,0,3,NaN,1,14.4583,0
...,...,...,...,...,...,...
835,1,1,39.0,1,83.1583,0
192,1,3,19.0,1,7.8542,0
629,0,3,NaN,0,7.7333,1
559,1,3,36.0,1,17.4000,0


## Ordinal-encoding

Заполенение категорий порядковым значением, если этот порядок нужен

In [5]:
ord_enc = ce.OrdinalEncoder(cols=['Sex']).fit(X_train,y_train)

In [6]:
ord_enc.transform(X_train)

,Survived,Pclass,Sex,Age,SibSp,Fare
857,1,1,1,51.0,0,26.5500
52,1,1,2,49.0,1,76.7292
386,0,3,1,1.0,5,46.9000
124,0,1,1,54.0,0,77.2875
578,0,3,2,NaN,1,14.4583
...,...,...,...,...,...,...
835,1,1,2,39.0,1,83.1583
192,1,3,2,19.0,1,7.8542
629,0,3,1,NaN,0,7.7333
559,1,3,2,36.0,1,17.4000


## Mean encoding

Замена категории средним значением целевой переменной для этой категории. При тестировании используем значения трейна. 

In [7]:
# Оценка среднего целевой переменной для пола
X_train['Survived'].groupby(data['Sex']).mean()

Sex
female    0.753488
male      0.196078
Name: Survived, dtype: float64

In [8]:
mean_enc = encoding.MeanEncoding(cols=['Sex']).fit(X_train,y_train)

In [9]:
mean_enc.transform(X_train)

,Survived,Pclass,Sex,Age,SibSp,Fare
857,1,1,0.196078,51.0,0,26.5500
52,1,1,0.753488,49.0,1,76.7292
386,0,3,0.196078,1.0,5,46.9000
124,0,1,0.196078,54.0,0,77.2875
578,0,3,0.753488,NaN,1,14.4583
...,...,...,...,...,...,...
835,1,1,0.753488,39.0,1,83.1583
192,1,3,0.753488,19.0,1,7.8542
629,0,3,0.196078,NaN,0,7.7333
559,1,3,0.753488,36.0,1,17.4000


## WOE-encoding

$WoE = log(p(X=x_j|Y=1)/(p(X=x_j|Y=0))$


* Если значения WoE отрицательны, отрицательные случаи превосходят положительные случаи для этой категории.
* Если значения WoE положительны, положительные случаи превосходят отрицательные случаи для этой категории.
* А если WoE равен 0, то положительных и отрицательных классов для этой категории поровну.

Если работаем с multi-class - то для каждого признака вычисляется WoE по статегии one-vs-all

In [10]:
woe_enc = ce.WOEEncoder(cols=['Sex']).fit(X_train,y_train)

In [11]:
woe_enc.transform(X_train)

,Survived,Pclass,Sex,Age,SibSp,Fare
857,1,1,-0.950742,51.0,0,26.5500
52,1,1,1.555633,49.0,1,76.7292
386,0,3,-0.950742,1.0,5,46.9000
124,0,1,-0.950742,54.0,0,77.2875
578,0,3,1.555633,NaN,1,14.4583
...,...,...,...,...,...,...
835,1,1,1.555633,39.0,1,83.1583
192,1,3,1.555633,19.0,1,7.8542
629,0,3,-0.950742,NaN,0,7.7333
559,1,3,1.555633,36.0,1,17.4000


### Замечания

* Если мы используем one-hot в линейной регрессии, нам следует сохранить k-1 one-hot признаков, чтобы избежать мультиколлинеарности. Это справедливо для любых ML алгоритмов, которые используют все признаки во время обучения - SVM, нейронные сети и алгоритмы кластеризации. С другой стороны, древовидному алгоритму необходим весь набор one-hot признаков для выбора наилучшего разделения.
* Но не рекомендуется использовать горячее кодирование с древовидными алгоритмами. One-hot приведет к тому, что разделение будет сильно несбалансированным (поскольку каждая метка исходного категориального признака теперь будет новым признаком), и в результате ни один из двух дочерних узлов не будет иметь значительного выигрыша в чистоте. Прогностическая способность one-hot признака будет слабее, чем у исходного признака, поскольку они разбиты на множество частей.